In [1]:
# imports
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import timezone
from datetime import datetime as dt
import numpy as np
import sys
from scipy.stats import pearsonr
from scipy import stats

sys.path.append("../")
from src.utils import *

In [2]:
# polars config
pl.Config.load_from_file("../polars_cfg.json")

# 1. Data loading

Choose whether to include participants that didn't complete all five tasks in the analyses with the True / False toggle.


In [3]:
# data file paths
clean_data_path = "../data/data.csv"

In [4]:
df_all = pl.read_csv(clean_data_path)

In [5]:
df_all.head()

exp_name,task_type,hashed_id,joint_id_task,task_name,task_number,is_tutorial,time,attempt_number,action_id,solved,done,test_input_grid,test_input_size_x,test_input_size_y,test_output_grid,test_output_size_x,test_output_size_y,action,action_x,action_y,select_loc,selected_data,selected_symbol,selected_tool,copy_paste_data,first_written_solution,last_written_solution,withdraw,withdraw_reason,withdraw_comment,age,gender,race,education_level,household_income,normal_vision,color_blind,fluent_english,complete
str,str,str,str,str,i64,bool,str,i64,i64,bool,bool,str,i64,i64,str,i64,i64,str,f64,f64,str,str,i64,str,str,str,str,bool,str,str,str,str,str,str,str,str,str,str,bool
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,false,"""2023-08-03T04:22:11.000000""",1,1,false,true,"""|000|000|000|""",3,3,"""|000|000|000|""",3,3,"""reset_grid""",null,null,null,"""[]""",0,"""edit""","""[]""","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",false,null,null,null,null,null,null,null,null,null,null,true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,false,"""2023-08-03T04:22:34.000000""",1,2,false,true,"""|10010010010010010|01001001001001001|00100100…",17,17,"""|000|000|000|""",3,3,"""change_color""",null,null,null,"""[]""",1,"""edit""","""[]""","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",false,null,null,null,null,null,null,null,null,null,null,true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,false,"""2023-08-03T04:22:46.000000""",1,3,false,true,"""|10010010010010010|01001001001001001|00100100…",17,17,"""|000|000|000|000|000|000|000|000|000|000|000|…",17,3,"""change_height""",null,null,null,"""[]""",1,"""edit""","""[]""","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",false,null,null,null,null,null,null,null,null,null,null,true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,false,"""2023-08-03T04:23:01.000000""",1,4,false,true,"""|10010010010010010|01001001001001001|00100100…",17,17,"""|00000000000000000|00000000000000000|00000000…",17,17,"""change_width""",null,null,null,"""[]""",1,"""edit""","""[]""","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",false,null,null,null,null,null,null,null,null,null,null,true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,false,"""2023-08-03T04:23:41.000000""",1,5,false,true,"""|10010010010010010|01001001001001001|00100100…",17,17,"""|00000000000000000|00000000000000000|00000000…",17,17,"""change_color""",null,null,null,"""[]""",1,"""edit""","""[]""","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",false,null,null,null,null,null,null,null,null,null,null,true


In [15]:
sorted_df = df_all.sort(
    by=["exp_name", "joint_id_task", "task_number", "attempt_number", "action_id"]
)
aggregated_df = sorted_df.group_by(
    "joint_id_task", "attempt_number", maintain_order=True
).agg(pl.col("test_input_grid").alias("grid_states"))

In [16]:
aggregated_df.head()

joint_id_task,attempt_number,grid_states
str,i64,list[str]
"""0017a2904e34497d578228a0912f2367_1caeab9d.jso…",1,"[""|000|000|000|"", ""|0000000022|0000000022|0110000200|0110000000|1000000000|0000440000|0000440000|0004000000|0000000000|0000000000|"", … ""|0000000022|0000000022|0110000200|0110000000|1000000000|0000440000|0000440000|0004000000|0000000000|0000000000|""]"
"""0017a2904e34497d578228a0912f2367_1e0a9b12.jso…",1,"[""|000|000|000|"", ""|02043|50000|00600|52040|50000|"", … ""|02043|50000|00600|52040|50000|""]"
"""0017a2904e34497d578228a0912f2367_1f876c06.jso…",1,"[""|000|000|000|"", ""|0000003009|7000000000|0000000000|0003009000|6000000000|0000000004|0000000000|0006007000|0000000000|0000040000|"", … ""|0000003009|7000000000|0000000000|0003009000|6000000000|0000000004|0000000000|0006007000|0000000000|0000040000|""]"
"""0017a2904e34497d578228a0912f2367_1fad071e.jso…",1,"[""|000|000|000|"", ""|000002201|110102200|110000000|000001100|022001100|022000000|100000220|220110220|220110000|"", … ""|000002201|110102200|110000000|000001100|022001100|022000000|100000220|220110220|220110000|""]"
"""0017a2904e34497d578228a0912f2367_3af2c5a8.jso…",1,"[""|000|000|000|"", ""|4000|0004|4400|"", … ""|4000|0004|4400|""]"


In [17]:
output_json_path = "../data/grid_states.json"

# Write the aggregated DataFrame to a JSON file
aggregated_df.write_json(output_json_path)